This is an example of coding LeNeT, one of the first CNNs

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Model, Sequential
import numpy as np

In [2]:
print(tf.__version__)
print(tf.keras.__version__)

2.3.0
2.4.0


Start by prepping the data

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [5]:
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

In [6]:
img_height = 28
img_width = 28
img_channels = 1
input_shape = (img_height, img_width, img_channels)

In [7]:
model = Sequential()
model.add(Conv2D(6, kernel_size=5, padding='valid', input_shape=(img_height,img_width, img_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=5, padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

In [8]:
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               30840     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [10]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'), 
            tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1)]
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')]

In [11]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(x=x_train, y=y_train, batch_size=32, epochs=4, validation_data=(x_test, y_test), callbacks=callbacks) 

Epoch 1/4
1875/1875 [==============================] - 11s 6ms/step - loss: 0.6587 - accuracy: 0.7890 - val_loss: 0.1740 - val_accuracy: 0.9467
Epoch 2/4
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1562 - accuracy: 0.9523 - val_loss: 0.1072 - val_accuracy: 0.9689
Epoch 3/4
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1082 - accuracy: 0.9669 - val_loss: 0.0864 - val_accuracy: 0.9742
Epoch 4/4
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0861 - accuracy: 0.9734 - val_loss: 0.0718 - val_accuracy: 0.9780


In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [14]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [15]:
model.fit(train_dataset, epochs=4, validation_data=test_dataset, callbacks=callbacks) 

Epoch 1/4
938/938 [==============================] - 7s 8ms/step - loss: 0.0658 - accuracy: 0.9794 - val_loss: 0.0628 - val_accuracy: 0.9798
Epoch 2/4
938/938 [==============================] - 8s 8ms/step - loss: 0.0609 - accuracy: 0.9809 - val_loss: 0.0595 - val_accuracy: 0.9807
Epoch 3/4
938/938 [==============================] - 8s 8ms/step - loss: 0.0566 - accuracy: 0.9822 - val_loss: 0.0621 - val_accuracy: 0.9796
Epoch 4/4
938/938 [==============================] - 7s 7ms/step - loss: 0.0529 - accuracy: 0.9834 - val_loss: 0.0563 - val_accuracy: 0.9819


Can also inherit from Model like so:

In [16]:
class LeNet(Model):
    def __init__(self, num_classes):
        super(LeNet, self).__init__()
        self.conv1 = Conv2D(6, kernel_size=5, padding='valid', input_shape=(img_height,img_width, img_channels), activation='relu')
        self.conv2 = Conv2D(16, kernel_size=5, padding='valid', activation='relu')
        self.max_pool = MaxPooling2D(pool_size=(2,2), strides=2)
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu')
        self.dense2 = Dense(84, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')
    def call(self, x):
        x = self.conv1(x)
        x = self.max_pool(x)
        x = self.conv2(x)
        x = self.max_pool(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

In [17]:
model = LeNet(10)

In [18]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'), 
            tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1)]
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')]

using test set as val!!!

In [20]:
model.fit(x=x_train, y=y_train, batch_size=32, epochs=4, validation_data=(x_test, y_test), callbacks=callbacks) 

Epoch 1/4
1875/1875 [==============================] - 11s 6ms/step - loss: 0.5958 - accuracy: 0.8090 - val_loss: 0.1809 - val_accuracy: 0.9446
Epoch 2/4
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1528 - accuracy: 0.9531 - val_loss: 0.1009 - val_accuracy: 0.9697
Epoch 3/4
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1062 - accuracy: 0.9665 - val_loss: 0.0763 - val_accuracy: 0.9763
Epoch 4/4
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0840 - accuracy: 0.9740 - val_loss: 0.0661 - val_accuracy: 0.9788


In [21]:
model.fit(x=x_train, y=y_train, batch_size=32, epochs=4, validation_data=(x_test, y_test), callbacks=callbacks) 

Epoch 1/4
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0703 - accuracy: 0.9779 - val_loss: 0.0619 - val_accuracy: 0.9793
Epoch 2/4
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0614 - accuracy: 0.9808 - val_loss: 0.0491 - val_accuracy: 0.9833
Epoch 3/4
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0542 - accuracy: 0.9826 - val_loss: 0.0482 - val_accuracy: 0.9848
Epoch 4/4
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0494 - accuracy: 0.9845 - val_loss: 0.0465 - val_accuracy: 0.9855


In [22]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape(x_train.shape[0], *input_shape)
x_test = x_test.reshape(x_test.shape[0], *input_shape)

In [23]:
class LeNet5(Model):
    
    def __init__(self, num_classes):
        """
        Initialize the model.
        :param num_classes:     Number of classes to predict from
        """
        super(LeNet5, self).__init__()
        # We instantiate the various layers composing LeNet-5:
        # self.conv1 = SimpleConvolutionLayer(6, kernel_size=(5, 5))
        # self.conv2 = SimpleConvolutionLayer(16, kernel_size=(5, 5))
        # ... or using the existing and (recommended) Conv2D class:
        self.conv1 = Conv2D(6, kernel_size=(5, 5), padding='same', activation='relu')
        self.conv2 = Conv2D(16, kernel_size=(5, 5), activation='relu')
        self.max_pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu')
        self.dense2 = Dense(84, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        """
        Call the layers and perform their operations on the input tensors
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        x = self.max_pool(self.conv1(inputs))        # 1st block
        x = self.max_pool(self.conv2(x))             # 2nd block
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x))) # dense layers
        return x

In [25]:
num_classes = 10

In [26]:
model = LeNet5(num_classes)
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [27]:

# We can call `model.summary()` only if the model was built before. 
# It is normally done automatically at the first use of the network,
# inferring the input shapes from the samples the network is given.
# For instance, the command below would build the network (then use it for prediction):
_ = model.predict(x_test[:10])

# But we can build the model manually otherwise, providing the batched
# input shape ourselves:
batched_input_shape = tf.TensorShape((None, *input_shape))
model.build(input_shape=batched_input_shape)

# Method to visualize the architecture of the network:
model.summary()

Model: "le_net5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            multiple                  156       
_________________________________________________________________
conv2d_5 (Conv2D)            multiple                  2416      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_2 (Flatten)          multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  48120     
_________________________________________________________________
dense_7 (Dense)              multiple                  10164     
_________________________________________________________________
dense_8 (Dense)              multiple                  850 

In [28]:
callbacks = [
    # Callback to interrupt the training if the validation loss (`val_loss`) stops improving for over 3 epochs:
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='accuracy'),
    # Callback to log the graph, losses and metrics into TensorBoard (saving log files in `./logs` directory):
    tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True)]

In [29]:
history = model.fit(x_train, y_train,
                    batch_size=32, epochs=80, validation_data=(x_test, y_test), 
                    verbose=2,  # change to `verbose=1` to get a progress bar
                                # (we opt for `verbose=2` here to reduce the log size)
                    callbacks=callbacks)

Epoch 1/80
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1875/1875 - 11s - loss: 0.5069 - accuracy: 0.8444 - val_loss: 0.1755 - val_accuracy: 0.9447
Epoch 2/80
1875/1875 - 11s - loss: 0.1330 - accuracy: 0.9586 - val_loss: 0.0961 - val_accuracy: 0.9706
Epoch 3/80
1875/1875 - 11s - loss: 0.0929 - accuracy: 0.9716 - val_loss: 0.0899 - val_accuracy: 0.9719
Epoch 4/80


KeyboardInterrupt: 